In [ ]:
# 1. サンプルデータセットをダウンロード
%cd /content
!git clone https://github.com/wal-afk/yolo_sample
%cd yolo_sample
!git pull
%cd /content

In [ ]:
# 2. データセット中のラベルの集計を表示
from yolo_sample.util.dataset_util import DatasetChecker

TARGET = f"yolo_sample/dataset/sample1"

ds = DatasetChecker(TARGET)
ds.print_labels_count()
ds.relocate_label_files()
ds.create_custom_yaml()

In [ ]:
# 2(補足). データセットの画像をいくつか表示 
ds.show_images_for_each_label()

In [ ]:
# 3. 学習用のライブラリをインストール
%pip list | grep torch

%pip install -q 'ultralytics<=8.3.162' 'onnxslim==0.1.57' 'onnx>=1.12.0,<1.18.0' 'onnxruntime-gpu'

!yolo settings tensorboard=True > /dev/null
%load_ext tensorboard
%pip list | grep ultralytics

In [ ]:
# 4. 学習の実行
import ultralytics
from ultralytics import YOLO

ultralytics.checks()
model = YOLO('yolo11n.pt') # load a pretrained model
train_result = model.train(data=f'{TARGET}/custom.yaml', exist_ok=True, epochs=60)

In [ ]:
# 5. 学習結果の評価
val_result=model.val(data=f'{TARGET}/custom.yaml')

In [ ]:
# 6. 学習したモデルを使用した物体認識
from glob import glob

from yolo_sample.util.show_util import show_all_images

results = model.predict(glob(f"{TARGET}/images/val/*.jpg"), verbose=False)

show_all_images([r.plot()[...,::-1] for r in results])

In [ ]:
# 7. 学習済みモデルの出力
path_onnx = model.export(format="onnx")
!ls runs/detect/train/weights -l

In [ ]:
# 8. 学習済みモデルのダウンロード
from google.colab import files

files.download(path_onnx)

In [ ]:
# 参考：　学習済みモデル原本のダウンロード

files.download("runs/detect/train/weights/best.pt")

In [ ]:
# 参考：学習時の評価で使用された画像の表示
from yolo_sample.util.show_util import show_2_images_path
for path in glob("runs/detect/train/val_batch*_labels.jpg"):
  show_2_images_path(path, path[:-len("labels.jpg")]+"pred.jpg")

In [ ]:
# 参考：学習結果の確認ツールの起動
%tensorboard --logdir runs